In [5]:
import os
import pandas as pd
from collections import defaultdict

# Папка с результатами
chatgpt_dir = '../ai-result/chatgpt'
files = [f for f in os.listdir(chatgpt_dir) if f.endswith('.txt')]

# Словарь: имя файла -> список персональных данных
file_to_pd = {}
all_pd = set()

for fname in files:
    path = os.path.join(chatgpt_dir, fname)
    try:
        with open(path, 'r', encoding='utf-8') as f:
            line = f.read().strip()
    except UnicodeDecodeError:
        # Пробуем cp1251, если utf-8 не сработал
        try:
            with open(path, 'r', encoding='cp1251') as f:
                line = f.read().strip()
        except Exception as e:
            print(f"Не удалось прочитать файл {fname}: {e}")
            line = ""
    except Exception as e:
        print(f"Не удалось прочитать файл {fname}: {e}")
        line = ""
    
    if line.startswith('Распознано:'):
        # Убираем 'Распознано:', разделяем по запятой, чистим пробелы и точки
        pd_list = [x.strip().rstrip('.') for x in line[len('Распознано:'):].split(',')]
        file_to_pd[fname] = pd_list
        all_pd.update(pd_list)
    elif 'Личных данных нет' in line:
        file_to_pd[fname] = ['Личных данных нет.']
        all_pd.add('Личных данных нет.')
    else:
        file_to_pd[fname] = []

# Сортируем для красивого вывода
all_pd_sorted = sorted(list(all_pd))

# Формируем датафрейм: строки - типы ПД, столбцы - файлы
df = pd.DataFrame(False, index=all_pd_sorted, columns=files)

# Заполняем датафрейм значениями True/False
for fname, pd_list in file_to_pd.items():
    for pd_item in pd_list:
        if pd_item in df.index:
            df.at[pd_item, fname] = True

# Транспонируем, чтобы файлы были строками, а типы ПД - столбцами
df_final = df.T

print("Размер итогового датасета:", df_final.shape)
df_final.head()

Размер итогового датасета: (55, 17)


,Адрес,Адрес проживания,Дата,ИНЗ,Имя,Инициалы,Инициалы врача,Личных данных нет.,Название организации,Населенный пункт,Номер пациента,Отчество,Отчество врача,Телефон,ФИО врача,Фамилия,Фамилия врача
9056305_depersonalized_chatgpt.txt,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False
9061455_depersonalized_chatgpt.txt,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False
9061458_depersonalized_chatgpt.txt,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False
9061491_depersonalized_chatgpt.txt,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False
9061527_depersonalized_chatgpt.txt,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False


In [6]:
# Преобразуем True/False в 1/0 для датафрейма df_final
df_numeric = df_final.astype(int)
df_numeric.head()

,Адрес,Адрес проживания,Дата,ИНЗ,Имя,Инициалы,Инициалы врача,Личных данных нет.,Название организации,Населенный пункт,Номер пациента,Отчество,Отчество врача,Телефон,ФИО врача,Фамилия,Фамилия врача
9056305_depersonalized_chatgpt.txt,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
9061455_depersonalized_chatgpt.txt,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
9061458_depersonalized_chatgpt.txt,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0
9061491_depersonalized_chatgpt.txt,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0
9061527_depersonalized_chatgpt.txt,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0


In [10]:
# В виде таблицы
# Считаем сумму по каждому параметру (столбцу)
param_sums = df_numeric.sum(axis=0)
param_sums_df = pd.DataFrame({'Параметр': param_sums.index, 'Сумма': param_sums.values})
display(param_sums_df)

,Параметр,Сумма
0,Адрес,11
1,Адрес проживания,1
2,Дата,2
3,ИНЗ,1
4,Имя,35
5,Инициалы,6
6,Инициалы врача,1
7,Личных данных нет.,8
8,Название организации,1
9,Населенный пункт,1


In [ ]:
import matplotlib.pyplot as plt

# Получаем суммы по каждому параметру
param_sums = df_numeric.sum(axis=0)

# Определяем цвета: "Личных данных нет." — красный, остальные — синий
colors = ['red' if col == 'Личных данных нет.' else 'steelblue' for col in param_sums.index]

plt.figure(figsize=(12, 6))
bars = plt.bar(param_sums.index, param_sums.values, color=colors)

plt.title('Частота встречаемости персональных данных в файлах')
plt.ylabel('Количество файлов')
plt.xlabel('Тип персональных данных')
plt.xticks(rotation=45, ha='right')

# Подписи над столбцами
for bar in bars:
    height = bar.get_height()
    plt.annotate(f'{int(height)}',
                 xy=(bar.get_x() + bar.get_width() / 2, height),
                 xytext=(0, 3),  # смещение подписи вверх
                 textcoords="offset points",
                 ha='center', va='bottom')

plt.tight_layout()
plt.show()